In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import validation_curve
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.model_selection import cross_val_score, train_test_split

In [2]:
%pwd

'/Users/ryanmaki'

In [5]:
bank_bal=pd.read_csv('OBA_410_Group_Data.csv')
bank_bal

,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
0,0,0.000000,0.006923,0.000000,0.593501,0.593551,0.998775,0.795533,0.807547,0.300687,...,0.224792,2.090270e-04,0.623797,0.593502,0.770902,0.285511,0.026790,0.565152,1,0.019171
1,1,0.024277,0.000000,0.033514,0.591324,0.591324,0.994015,0.791503,0.803775,0.303602,...,0.584583,2.710000e+09,0.623818,0.591322,0.827165,0.276846,0.026791,0.565158,1,0.054959
2,1,0.066933,0.057185,0.054821,0.601861,0.601861,0.998825,0.796779,0.808717,0.302760,...,0.525651,5.803037e-03,0.623648,0.601857,1.000000,0.182790,0.026763,0.565021,1,0.009178
3,0,0.081412,0.091256,0.091761,0.602935,0.602935,0.973875,0.767759,0.781429,0.304248,...,0.412621,1.920000e+09,0.623783,0.602932,0.813135,0.278182,0.026791,0.565158,1,0.037886
4,0,0.082826,0.069287,0.088388,0.646846,0.646846,0.997083,0.795795,0.807815,0.304684,...,0.000000,0.000000e+00,0.623730,0.646841,0.748252,0.284685,0.026791,0.565158,1,0.019908
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6814,0,0.787988,0.804514,0.783928,0.608015,0.608015,0.999147,0.798222,0.809903,0.304606,...,0.905790,2.948022e-03,0.624791,0.608017,0.847232,0.277502,0.026791,0.565159,1,0.044549
6815,0,0.818018,0.801570,0.814551,0.634983,0.634983,0.999278,0.797758,0.809581,0.303522,...,0.923155,1.093630e-04,0.623786,0.634983,0.848965,0.278178,0.026794,0.565170,1,0.037918
6816,0,0.864964,0.942706,0.932598,0.622371,0.622371,0.999308,0.797793,0.809673,0.303519,...,0.959320,2.271900e-04,0.624001,0.622370,0.849236,0.276178,0.026791,0.565158,1,0.074893
6817,0,0.971530,1.000000,1.000000,0.640035,0.640035,0.999571,0.798105,0.809906,0.303516,...,0.981315,1.534960e-04,0.624305,0.640035,0.851547,0.277013,0.026799,0.565196,1,0.051752


In [6]:
# bank_bal.drop(bank_bal.iloc[:,10:], inplace=True, axis=1)

In [7]:
bank_bal=bank_bal.groupby('Bankrupt?',group_keys=False).apply(lambda x: x.sample(1000,replace=True,random_state=0))
bank_bal['Bankrupt?'].value_counts()

0    1000
1    1000
Name: Bankrupt?, dtype: int64

In [8]:
bank_bal

,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
2935,0,0.495686,0.550916,0.542963,0.599209,0.599209,0.999001,0.797404,0.809320,0.303483,...,0.804195,5.797642e-03,0.623865,0.599205,0.840688,0.279388,0.027243,0.566668,1,0.030801
2807,0,0.494028,0.556149,0.539697,0.603273,0.603021,0.999016,0.797448,0.809353,0.303527,...,0.806766,1.678171e-03,0.623481,0.603273,0.841082,0.281947,0.027068,0.566204,1,0.023555
1834,0,0.478672,0.541976,0.531881,0.605226,0.605226,0.999008,0.797419,0.809341,0.303494,...,0.801978,4.771583e-03,0.623800,0.605225,0.840553,0.280316,0.026973,0.565898,1,0.027411
3475,0,0.503924,0.551297,0.549280,0.607064,0.607064,0.998987,0.797522,0.809440,0.303718,...,0.804269,4.786887e-02,0.624181,0.607060,0.840859,0.282237,0.027089,0.566266,1,0.023043
5150,0,0.536635,0.597580,0.580813,0.610466,0.610141,0.999138,0.797629,0.809496,0.303588,...,0.832874,3.267898e-03,0.625247,0.610468,0.842125,0.276202,0.026792,0.565160,1,0.073883
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
928,1,0.454346,0.513792,0.506023,0.597522,0.597522,0.998899,0.797237,0.809064,0.303405,...,0.771996,6.000143e-03,0.621979,0.597522,0.837544,0.283088,0.026463,0.562888,1,0.021747
1791,1,0.477990,0.538323,0.529632,0.616966,0.616930,0.999190,0.797473,0.809361,0.303207,...,0.797931,2.171998e-02,0.619427,0.616962,0.840159,0.276686,0.027789,0.567605,1,0.058527
368,1,0.417540,0.470726,0.460892,0.600852,0.600852,0.998935,0.797160,0.809086,0.303195,...,0.754696,8.433273e-03,0.624491,0.600848,0.836693,0.279759,0.026550,0.563668,1,0.029296
1250,1,0.466728,0.490078,0.501097,0.607850,0.607850,0.998855,0.797196,0.809109,0.303425,...,0.767057,8.094303e-03,0.623321,0.607845,0.834982,0.293298,0.026485,0.563103,1,0.015432


# Logistic Regression¶

In [9]:
bank_bal[bank_bal['Bankrupt?']==1].describe().round(2)

,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
count,1000.0,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,...,1000.00,1.000000e+03,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.0,1000.00
mean,1.0,0.42,0.46,0.46,0.60,0.60,1.00,0.80,0.81,0.30,...,0.74,5.630000e+07,0.62,0.60,0.83,0.29,0.03,0.56,1.0,0.02
std,0.0,0.08,0.11,0.09,0.01,0.01,0.00,0.00,0.00,0.01,...,0.08,5.971907e+08,0.01,0.01,0.06,0.05,0.02,0.01,0.0,0.05
min,1.0,0.02,0.00,0.03,0.53,0.53,0.99,0.76,0.76,0.24,...,0.41,0.000000e+00,0.59,0.53,0.00,0.00,0.02,0.55,1.0,0.00
25%,1.0,0.39,0.43,0.44,0.59,0.59,1.00,0.80,0.81,0.30,...,0.72,0.000000e+00,0.62,0.59,0.83,0.28,0.03,0.56,1.0,0.02
50%,1.0,0.44,0.49,0.49,0.60,0.60,1.00,0.80,0.81,0.30,...,0.77,0.000000e+00,0.62,0.60,0.84,0.29,0.03,0.56,1.0,0.02
75%,1.0,0.47,0.53,0.52,0.60,0.60,1.00,0.80,0.81,0.30,...,0.79,1.000000e-02,0.62,0.60,0.84,0.29,0.03,0.57,1.0,0.02
max,1.0,0.58,0.58,0.62,0.67,0.67,1.00,0.80,0.81,0.31,...,0.82,9.170000e+09,0.68,0.67,1.00,1.00,0.26,0.62,1.0,0.80


In [10]:
bank_bal[bank_bal['Bankrupt?']==0].describe().round(2)

,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
count,1000.0,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,...,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.0,1000.00
mean,0.0,0.51,0.56,0.56,0.61,0.61,1.00,0.80,0.81,0.30,...,0.81,0.01,0.62,0.61,0.84,0.28,0.03,0.57,1.0,0.05
std,0.0,0.06,0.06,0.06,0.01,0.01,0.01,0.02,0.03,0.01,...,0.04,0.04,0.01,0.01,0.00,0.01,0.02,0.01,0.0,0.06
min,0.0,0.00,0.01,0.00,0.45,0.45,0.61,0.16,0.00,0.00,...,0.22,0.00,0.55,0.45,0.77,0.27,0.00,0.33,1.0,0.01
25%,0.0,0.48,0.54,0.53,0.60,0.60,1.00,0.80,0.81,0.30,...,0.80,0.00,0.62,0.60,0.84,0.28,0.03,0.57,1.0,0.03
50%,0.0,0.50,0.56,0.55,0.61,0.61,1.00,0.80,0.81,0.30,...,0.81,0.00,0.62,0.61,0.84,0.28,0.03,0.57,1.0,0.04
75%,0.0,0.54,0.59,0.59,0.61,0.61,1.00,0.80,0.81,0.30,...,0.83,0.01,0.62,0.61,0.84,0.28,0.03,0.57,1.0,0.06
max,0.0,1.00,0.95,0.96,0.66,0.66,1.00,0.81,0.82,0.33,...,0.94,0.54,0.70,0.66,0.85,0.33,0.54,0.74,1.0,1.00


In [11]:
X_bank, y_bank=bank_bal.iloc[:,1:], bank_bal['Bankrupt?'] 
display(X_bank.head(2))
display(y_bank.head(2))

,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,Continuous interest rate (after tax),...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
2935,0.495686,0.550916,0.542963,0.599209,0.599209,0.999001,0.797404,0.809320,0.303483,0.781574,...,0.804195,0.005798,0.623865,0.599205,0.840688,0.279388,0.027243,0.566668,1,0.030801
2807,0.494028,0.556149,0.539697,0.603273,0.603021,0.999016,0.797448,0.809353,0.303527,0.781613,...,0.806766,0.001678,0.623481,0.603273,0.841082,0.281947,0.027068,0.566204,1,0.023555


2935    0
2807    0
Name: Bankrupt?, dtype: int64

In [12]:
from sklearn.tree import DecisionTreeClassifier

In [13]:
dt=DecisionTreeClassifier(random_state=0)
dt.fit(X_bank,y_bank)

DecisionTreeClassifier(random_state=0)

In [14]:
sum(dt.feature_importances_>.01)

16

In [15]:
X_bank_sel=X_bank[X_bank.columns[dt.feature_importances_>.01]]
X_bank_sel.head()

,Continuous interest rate (after tax),Operating Expense Rate,Interest-bearing debt interest rate,Net Value Per Share (A),Persistent EPS in the Last Four Seasons,Operating Profit Growth Rate,Total Asset Growth Rate,Quick Ratio,Interest Expense Ratio,Total debt/Total net worth,Accounts Receivable Turnover,Operating profit per person,Allocation rate per person,Cash Turnover Rate,Cash Flow to Sales,Interest Coverage Ratio (Interest expense to EBIT)
2935,0.781574,5.820000e+09,0.000672,0.177447,0.217831,0.848084,5.550000e+09,0.007517,0.632188,0.006354,0.001469,0.396373,0.005721,2.580000e+08,0.671572,0.566668
2807,0.781613,1.038420e-04,0.000359,0.190342,0.224355,0.848025,8.000000e+09,0.003586,0.631575,0.009883,0.001108,0.398295,0.006320,6.650000e+09,0.671579,0.566204
1834,0.781601,1.474370e-04,0.000141,0.171421,0.218020,0.848012,5.830000e+09,0.006185,0.631163,0.007635,0.000363,0.394677,0.015585,1.250000e+09,0.671575,0.565898
3475,0.781594,1.973910e-04,0.000484,0.189415,0.216129,0.848184,1.039850e-04,0.004459,0.632241,0.010284,0.002682,0.392879,0.048975,2.072150e-04,0.671454,0.566266
5150,0.781749,9.560000e+09,0.000000,0.234419,0.245722,0.848215,7.490000e+09,0.022148,0.630614,0.001962,0.001175,0.403247,0.002276,2.695690e-04,0.671648,0.565160


In [16]:
scaler1=MinMaxScaler()
X_bank_trns=scaler1.fit_transform(X_bank_sel)
X_bank_trns

array([[9.78771560e-01, 5.87285570e-01, 7.00069792e-13, ...,
        2.58258258e-02, 9.72108922e-01, 5.79849651e-01],
       [9.78829637e-01, 3.85065590e-16, 3.73995833e-13, ...,
        6.65665666e-01, 9.72118950e-01, 5.78705083e-01],
       [9.78812680e-01, 4.78415742e-15, 1.46889583e-13, ...,
        1.25125125e-01, 9.72112531e-01, 5.77951033e-01],
       ...,
       [9.78397658e-01, 5.07578204e-15, 8.43834375e-13, ...,
        1.39725726e-14, 9.72129663e-01, 5.72450276e-01],
       [9.78439202e-01, 2.44808274e-14, 4.43794792e-13, ...,
        9.09909910e-02, 9.72104756e-01, 5.71057391e-01],
       [9.69751507e-01, 4.86501715e-13, 0.00000000e+00, ...,
        2.34577578e-14, 9.72031743e-01, 5.76126590e-01]])

In [17]:
X_train, X_test, y_train, y_test=train_test_split(X_bank_trns, y_bank, test_size=0.2, random_state=0)

In [18]:
train_scores_log_reg, test_scores_log_reg=validation_curve(LogisticRegression(penalty='l1', solver='saga', max_iter=100000), X_train, y_train, param_name='C',
                                           param_range=[0.01,0.05,1,5,10,20], cv=4)

In [19]:
train_scores_log_reg.mean(axis=1).round(4)

array([0.5   , 0.57  , 0.8023, 0.8071, 0.8088, 0.8106])

In [20]:
test_scores_log_reg.mean(axis=1).round(4)

array([0.5   , 0.5581, 0.7981, 0.8019, 0.8038, 0.8062])

In [21]:
log_reg=LogisticRegression(max_iter=1000, C=20)
log_reg.fit(X_train, y_train)

LogisticRegression(C=20, max_iter=1000)

In [22]:
print('log reg acc on train: {:.2%}'.format(log_reg.score(X_train, y_train)))
print('log reg acc on test: {:.2%}'.format(log_reg.score(X_test, y_test)))

log reg acc on train: 79.94%
log reg acc on test: 78.50%


In [23]:
log_reg.coef_

array([[ 1.80562301e+00,  6.58093325e-01, -2.48466435e+00,
        -1.85227806e+01, -3.48278375e+01, -3.57157720e+00,
        -8.00673043e-02,  2.90046551e-01,  1.33873194e+00,
         1.69666951e+00,  1.11453148e+00,  1.46028712e-02,
         9.83097328e-01, -9.80587826e-02,  1.42027076e+00,
         5.31704890e-01]])

In [24]:
p1=[0.2,0.4,0.5,0.1,0.4,0.3,0.2,0.5,0.2,0.5,0.5,0.2,0.2,0.4,0.5,0.2,0.1]
log_reg.predict([p1])

ValueError: X has 17 features, but LogisticRegression is expecting 16 features as input.

In [25]:
import numpy as np
np.set_printoptions(suppress=True)

In [26]:
log_reg.predict_proba([p1]).round(4)

ValueError: X has 17 features, but LogisticRegression is expecting 16 features as input.

In [27]:
log_reg.classes_

array([0, 1])

In [28]:
log_reg.coef_.round(4)

array([[  1.8056,   0.6581,  -2.4847, -18.5228, -34.8278,  -3.5716,
         -0.0801,   0.29  ,   1.3387,   1.6967,   1.1145,   0.0146,
          0.9831,  -0.0981,   1.4203,   0.5317]])

In [29]:
log_reg.intercept_.round(4)

array([9.9882])

# SVM 

In [30]:
svm1=LinearSVC(random_state=0)
svm1.fit(X_train,y_train)

LinearSVC(random_state=0)

In [31]:
svm1.coef_

array([[  1.22938622,   0.24493122,  -0.8032349 ,  -6.99867714,
        -10.02797319,  -0.5244424 ,  -0.07087926,   0.16478105,
          0.40563937,   0.46416572,   0.36308411,  -0.64152345,
          0.34479265,  -0.03438365,   1.31493451,   0.31815455]])

In [32]:
svm1.intercept_

array([1.43886974])

In [33]:
print('svm acc on train: {:.2%}'.format(svm1.score(X_train, y_train)))
print('svm acc on test: {:.2%}'.format(svm1.score(X_test, y_test)))

svm acc on train: 79.75%
svm acc on test: 78.25%


In [34]:
b=[[0.616,0.999,0.781,0.0001,0.471,0.0005,0.631,0.3696,0.0415,0.0022,0.27698,0,0.953,0,0.7022,0.0029,0.0345]]
svm1.predict(b)

ValueError: X has 17 features, but LinearSVC is expecting 16 features as input.

In [35]:
b_trns=scaler1.transform(b)
b_trns

/Users/ryanmaki/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


ValueError: X has 17 features, but MinMaxScaler is expecting 16 features as input.

In [36]:
svm1.predict((b_trns))

NameError: name 'b_trns' is not defined

In [37]:
svm1.intercept_.round(2)

array([1.44])

In [38]:
svm1.coef_.round(2)

array([[  1.23,   0.24,  -0.8 ,  -7.  , -10.03,  -0.52,  -0.07,   0.16,
          0.41,   0.46,   0.36,  -0.64,   0.34,  -0.03,   1.31,   0.32]])

In [39]:
LinearSVC()

LinearSVC()

In [40]:
C_range=[.1,1,5,10,50,100]

In [41]:
train_scores_SVM, test_scores_SVM=validation_curve(LinearSVC(random_state=0, max_iter=100000), X_bank_trns, y_bank, param_name='C',
                                           param_range=[.1,1,5,10,50,100], cv=4)

In [42]:
train_scores_SVM.round(4)

array([[0.752 , 0.7667, 0.7593, 0.7593],
       [0.7833, 0.79  , 0.7913, 0.7927],
       [0.8   , 0.8   , 0.8007, 0.804 ],
       [0.8033, 0.802 , 0.7947, 0.8   ],
       [0.81  , 0.8027, 0.7973, 0.8067],
       [0.81  , 0.8027, 0.7987, 0.8053]])

In [46]:
print('ave cross in 4 folds for different C values:')
print('training:', train_scores_SVM.mean(axis=1).round(4))
print('test', test_scores_SVM.mean(axis=1).round(4))

ave cross in 4 folds for different C values:
training: [0.7593 0.7893 0.8012 0.8    0.8042 0.8042]
test [0.7525 0.786  0.798  0.798  0.7995 0.799 ]


In [48]:
train_scores_SVM.mean()

0.7930277777777778

# Decision tree

In [49]:
X_train, X_test, y_train, y_test=train_test_split(X_bank_sel, y_bank, test_size=0.2, random_state=0)

In [50]:
X_train

,Continuous interest rate (after tax),Operating Expense Rate,Interest-bearing debt interest rate,Net Value Per Share (A),Persistent EPS in the Last Four Seasons,Operating Profit Growth Rate,Total Asset Growth Rate,Quick Ratio,Interest Expense Ratio,Total debt/Total net worth,Accounts Receivable Turnover,Operating profit per person,Allocation rate per person,Cash Turnover Rate,Cash Flow to Sales,Interest Coverage Ratio (Interest expense to EBIT)
1376,0.781566,9.310000e+09,0.000267,0.177279,0.214900,0.848122,8.880000e+09,0.005079,0.633328,0.021935,0.000706,0.390957,0.001681,2.230000e+09,0.671576,0.567550
6215,0.781836,1.455860e-04,0.000000,0.261304,0.284202,0.848040,7.410000e+09,0.012941,0.630617,0.003169,0.000611,0.405237,0.006409,1.566090e-04,0.671570,0.565164
349,0.781454,6.550000e+09,0.000201,0.153639,0.186064,0.848017,6.530000e+09,0.006014,0.630282,0.026291,0.000812,0.372267,0.007499,3.700000e+09,0.671568,0.564661
3911,0.781616,9.660000e+09,0.000826,0.177110,0.223598,0.848026,7.590000e+09,0.004494,0.631522,0.006069,0.001485,0.395920,0.009785,1.130000e+09,0.671573,0.566431
5464,0.781675,6.735830e-04,0.000000,0.180397,0.229744,0.848096,6.830000e+09,0.009013,0.630612,0.002608,0.001321,0.394677,0.002524,5.610000e+09,0.671568,0.565158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4945,0.781618,7.120000e+09,0.000927,0.207829,0.237024,0.850289,8.150000e+09,0.005253,0.631644,0.004416,0.002875,0.400669,0.007736,1.850000e+09,0.671570,0.566489
3158,0.781625,1.025700e-04,0.000647,0.178838,0.223787,0.848022,7.240000e+09,0.002789,0.632245,0.013788,0.001035,0.393829,0.013643,4.440000e+07,0.671573,0.567263
1924,0.781602,5.060000e+08,0.000481,0.193039,0.221802,0.848050,6.790000e+08,0.002089,0.633201,0.018054,0.000781,0.398171,0.002960,8.190000e+08,0.671565,0.567897
3357,0.781634,9.140000e+09,0.000206,0.174835,0.219438,0.848161,6.980000e+09,0.007633,0.630861,0.002505,0.000988,0.395186,0.010188,8.180000e+09,0.671574,0.565472


In [51]:
train_scores_decision_tree, test_scores_decision_tree=validation_curve(DecisionTreeClassifier(random_state=0), X_train, y_train, param_name='max_leaf_nodes', 
                                          param_range=[2,6,8,10,15,25,50], cv=5)

In [52]:
dt0=DecisionTreeClassifier(random_state=0, max_leaf_nodes=10)
dt0.fit(X_train, y_train)

DecisionTreeClassifier(max_leaf_nodes=10, random_state=0)

In [53]:
print('dt0 acc on train: {:.2%}'.format(dt0.score(X_train, y_train)))
print('dt0 acc on test: {:.2%}'.format(dt0.score(X_test, y_test)))

dt0 acc on train: 91.81%
dt0 acc on test: 86.25%


In [54]:
p1=[0.616,0.999,0.781,0.0001,0.471,0.0005,0.631,0.3696,0.0415,0.0022,0.27698,0,0.953,0,0.7022,0.0029,0.0345]
dt0.predict([p1])

/Users/ryanmaki/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


ValueError: X has 17 features, but DecisionTreeClassifier is expecting 16 features as input.

In [55]:
export_graphviz(dt0, out_file='dt0_vis.dot', feature_names=X_bank_sel.columns, class_names=dt0.classes_.astype(str), impurity=False, filled=True)

In [56]:
dt0.classes_

array([0, 1])

In [57]:
dt0.feature_importances_.round(3)

array([0.166, 0.   , 0.077, 0.   , 0.086, 0.   , 0.024, 0.   , 0.   ,
       0.581, 0.   , 0.022, 0.   , 0.   , 0.023, 0.022])

In [58]:
X_bank_sel.columns

Index([' Continuous interest rate (after tax)', ' Operating Expense Rate',
       ' Interest-bearing debt interest rate', ' Net Value Per Share (A)',
       ' Persistent EPS in the Last Four Seasons',
       ' Operating Profit Growth Rate', ' Total Asset Growth Rate',
       ' Quick Ratio', ' Interest Expense Ratio',
       ' Total debt/Total net worth', ' Accounts Receivable Turnover',
       ' Operating profit per person', ' Allocation rate per person',
       ' Cash Turnover Rate', ' Cash Flow to Sales',
       ' Interest Coverage Ratio (Interest expense to EBIT)'],
      dtype='object')

In [59]:
feat_imp=pd.DataFrame(data=dt0.feature_importances_.round(3),index=X_bank_sel.columns, columns=['importance'])
feat_imp.sort_values('importance', ascending=False)

,importance
Total debt/Total net worth,0.581
Continuous interest rate (after tax),0.166
Persistent EPS in the Last Four Seasons,0.086
Interest-bearing debt interest rate,0.077
Total Asset Growth Rate,0.024
Cash Flow to Sales,0.023
Operating profit per person,0.022
Interest Coverage Ratio (Interest expense to EBIT),0.022
Operating Expense Rate,0.000
Net Value Per Share (A),0.000


# KNN Classifier

In [60]:
train_scores_KNN, test_scores_KNN=validation_curve(KNeighborsClassifier(), X_bank_trns, y_bank, param_name='n_neighbors', 
                                           param_range=[1,2,3,4,5,6], cv=4)

In [61]:
train_scores_KNN.mean(axis=1)

array([1.        , 0.99      , 0.94016667, 0.92966667, 0.897     ,
       0.886     ])

In [62]:
train_scores_KNN.mean()

0.9404722222222223

In [63]:
test_scores_KNN.mean()

0.8690000000000001

In [64]:
X_train, X_test, y_train, y_test=train_test_split(X_bank_sel,y_bank,
                                                  random_state=0)

In [65]:
knn_cls_bank_bal=KNeighborsClassifier(n_neighbors=5)

In [66]:
knn_cls_bank_bal.fit(X_train,y_train)

KNeighborsClassifier()

In [67]:
print('knn_cls_bank_bal acc on train: {:.2%}'.format(knn_cls_bank_bal.score(X_train,y_train)))
print('knn_cls_bank_bal acc on test: {:.2%}'.format(knn_cls_bank_bal.score(X_test,y_test)),'\n')

knn_cls_bank_bal acc on train: 85.67%
knn_cls_bank_bal acc on test: 73.40% 



In [68]:
knn_cls_bank_bal.score(X_bank_sel,y_bank)

0.826

In [69]:
w1=[1,1,0.5,0.4,0.3,0.3,0.6,0.7,1,0.4,0.2,0.5,0.7,0.4,0.1,0.9,0.6]
knn_cls_bank_bal.predict([w1])

/Users/ryanmaki/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


ValueError: X has 17 features, but KNeighborsClassifier is expecting 16 features as input.

In [70]:
knn_cls_bank_bal.predict(X_bank_sel)

array([1, 0, 1, ..., 1, 0, 1])

In [71]:
len(knn_cls_bank_bal.predict(X_bank_sel))

2000

In [72]:
predict_array = knn_cls_bank_bal.predict(X_bank_sel)
count_non_zero = 0
for i in predict_array:
    if i != 0:
        count_non_zero += 1
print(count_non_zero)      

1198


# Fair Evaluation 

In [73]:
print('initial features set size:',X_bank_sel.shape)
print('X_train shape',X_train.shape)
print('X_test shape',X_test.shape)

print('y_train shape',y_train.shape)
print('y_test shape',y_test.shape)

initial features set size: (2000, 16)
X_train shape (1500, 16)
X_test shape (500, 16)
y_train shape (1500,)
y_test shape (500,)


In [74]:
knn_cls_bank_bal=KNeighborsClassifier(n_neighbors=5)

In [75]:
knn_cls_bank_bal.fit(X_train, y_train)

KNeighborsClassifier()

In [76]:
print('knn accuracy on training data:', knn_cls_bank_bal.score(X_train, y_train))
print('knn accuracy on test data:', knn_cls_bank_bal.score(X_test, y_test))

knn accuracy on training data: 0.8566666666666667
knn accuracy on test data: 0.734


# Cross-Validation

In [77]:
knn_cls_bank_bal1=KNeighborsClassifier(n_neighbors=9)

In [78]:
scores=cross_val_score(knn_cls_bank_bal1,X_bank_sel,y_bank,cv=6)
scores

array([0.76946108, 0.70359281, 0.72072072, 0.72372372, 0.67567568,
       0.69369369])

In [79]:
scores.mean()

0.7144779510048972

In [80]:
print('Average test accuracy in a 6-fold cross val is: {:.2%}'.format(
    scores.mean()))

Average test accuracy in a 6-fold cross val is: 71.45%


# Overfit and Underfit

In [81]:
X_train, X_test, y_train, y_test=train_test_split(X_bank_sel, y_bank, random_state=0, test_size=0.25)

In [82]:
knn1=KNeighborsClassifier(n_neighbors=1)
knn5=KNeighborsClassifier(n_neighbors=5)
knn9=KNeighborsClassifier(n_neighbors=9)
knn29=KNeighborsClassifier(n_neighbors=29)

In [83]:
knn1.fit(X_train, y_train)
knn5.fit(X_train, y_train)
knn9.fit(X_train, y_train)
knn29.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=29)

In [84]:
print('knn1 acc on train: {:.2%}'.format(knn1.score(X_train, y_train)))
print('knn1 acc on test: {:.2%}'.format(knn1.score(X_test, y_test)),'\n')

print('knn5 acc on train: {:.2%}'.format(knn5.score(X_train, y_train)))
print('knn5 acc on test: {:.2%}'.format(knn5.score(X_test, y_test)),'\n')

print('knn9 acc on train: {:.2%}'.format(knn9.score(X_train, y_train)))
print('knn9 acc on test: {:.2%}'.format(knn9.score(X_test, y_test)),'\n')

print('knn29 acc on train: {:.2%}'.format(knn29.score(X_train, y_train)))
print('knn29 acc on test: {:.2%}'.format(knn29.score(X_test, y_test)))

knn1 acc on train: 99.67%
knn1 acc on test: 92.80% 

knn5 acc on train: 85.67%
knn5 acc on test: 73.40% 

knn9 acc on train: 78.80%
knn9 acc on test: 67.80% 

knn29 acc on train: 73.40%
knn29 acc on test: 69.20%
